## **Librairies nécessaires à l'importation et aux traitements des données**

In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import polars as pl
import requests
from datetime import datetime
from pathlib import Path
from zipfile import ZipFile

In [2]:
# Il faut télécharger la base SIRENE 3.11 "Fichier StockEtablissement" le plus récent sur le site : https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/#/resources 

version = datetime.now().strftime("%B_%Y") # Version nous permet de dater l'import du fichier au mois et à l'année dans le soucis de constituer et stocker une base de données à long terme
version_jour = datetime.now().strftime("%d_%m_%Y")
path_sirene_zip = Path().home().joinpath(f"Documents\BDD sirene\sirene 3.11\StockEtablissement_{version}.zip") # On stock le fichier zip dans le répertoire de notre choix. Le chemin est donc à modifier selon l'utilisateur. 

# On télécharge le fichier zip par requête 
url_zip_sirene = "https://files.data.gouv.fr/insee-sirene/StockEtablissement_utf8.zip"
response = requests.get(url_zip_sirene, stream=True)

with open(path_sirene_zip, "wb") as file:
    for chunk in response.iter_content(chunk_size=1024):

        file.write(chunk) 

print("Téléchargement du fichier zip SIRENE 3.11 : effectué.")

# On extrait le fichier zip pour pouvor accèder au fichier csv
with ZipFile(path_sirene_zip, "r") as zip:
    zip.printdir()
    zip.extractall()

print("Dézippage du fichier SIRENE 3.11 : effectué.")

# On applique directement dans l'import les modifications pour obtenir les bons formats de variables et de valeurs nulles
stock = pl.read_csv(path_sirene_zip.parent.joinpath("StockEtablissement_utf8.csv"), schema_overrides={"dernierNumeroVoieEtablissement": pl.datatypes.String,
                                "numeroVoieEtablissement": pl.datatypes.String,
                                "codeCommuneEtablissement": pl.datatypes.String,
                                "codePostalEtablissement": pl.datatypes.String,},
              null_values=["[ND]", "NN"]
)

print("Importation du fichier SIRENE 3.11 au format CSV : effectué.")

# On utilise les codes APE pour selectionner les établissements qui nous intéressent à savoir les commerces alimentaires
code_etablissement = ["10.71B", "10.71C", "10.71D", "47.11A", "47.11B", "47.11C", "47.11D", "47.11F", "47.11E", "47.21Z", "47.22Z", "47.23Z", "47.24Z", "47.29Z", "56.10A", "56.10B", "56.10C"]
stock_commerce_alim = stock.filter((pl.col("activitePrincipaleEtablissement").is_in(code_etablissement)))

print("Filtre de la base SIRENE 3.11 sur les commerces alimentaires effectué.")

# On sélectionne les établissements actifs de la base
stock_commerce_alim_actif = stock_commerce_alim.filter((pl.col("etatAdministratifEtablissement") == "A"))

print("Filtre sur les commerces alimentaires actifs : effectué.")

stock_commerce_alim_actif = stock_commerce_alim_actif.to_pandas()

# Conversion de certaines variables
cols_object = ["siret", "siren", "nic", "anneeEffectifsEtablissement"]
stock_commerce_alim_actif[cols_object] = stock_commerce_alim_actif[cols_object].astype(object, errors='ignore')
stock_commerce_alim_actif =  stock_commerce_alim_actif.astype({"trancheEffectifsEtablissement": 'Int64'}, errors='ignore')



<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
C:\Users\qcreurer\AppData\Local\Temp\ipykernel_20184\227002037.py:5: SyntaxWarning: invalid escape sequence '\B'
  path_sirene_zip = Path().home().joinpath(f"Documents\BDD sirene\sirene 3.11\StockEtablissement_{version}.zip") # On stock le fichier zip dans le répertoire de notre choix. Le chemin est donc à modifier selon l'utilisateur.


Téléchargement du fichier zip SIRENE 3.11 : effectué.
File Name                                             Modified             Size
StockEtablissement_utf8.csv                    2025-02-03 12:14:00   8798569046
Dézippage du fichier SIRENE 3.11 : effectué.
Importation du fichier SIRENE 3.11 au format CSV : effectué.
Filtre de la base SIRENE 3.11 sur les commerces alimentaires effectué.
Filtre sur les commerces alimentaires actifs : effectué.


## **Apport d'Alim confiance dans la base de données SIRENE 3.11**

In [3]:
# On reprend le même principe qu'avec la base SIRENE, on va chercher le fichier alim-confiance par requête puis on le stock dans le même répertoire que SIRENE 3.11 

path_alim_confiance = Path().home().joinpath(f"Documents/BDD sirene/sirene 3.11/alim-confiance{version}.csv")
url_alim_confiance = "https://www.data.gouv.fr/fr/datasets/r/ab066871-ac7c-4e13-ae47-83a2548e2b54"

response_alim_confiance = requests.get(url_alim_confiance, stream=True)

with open(path_alim_confiance, "wb") as file:
    for chunk in response_alim_confiance.iter_content(chunk_size=1024):

        file.write(chunk) 

print("Téléchargement du fichier alim-confiance : effectué.")

# On importe le fichier téléchargé et des modifications sont nécessaires afin de pouvoir faire une jointure avec SIRENE 3.11
alim_confiance = pl.read_csv(path_alim_confiance.parent.joinpath(f"alim-confiance{version}.csv"), separator=";", ignore_errors=True)
alim_confiance = alim_confiance.to_pandas()

print("Importation du fichier CSV alim-confiance : effectué.")

alim_confiance = alim_confiance.drop_duplicates() # Traitements des entités doublons
alim_confiance = alim_confiance.rename(columns={"SIRET" : "siret"}) # Renommer la variable siret selon la typologie de SIRENE 3.11
stock_commerce_alim_actif_alim_confiance = pd.merge(stock_commerce_alim_actif, alim_confiance, on="siret", how="left") # Fusionner les deux fichiers

print("Jointure entre le fichier des commerces alimentaires actifs et alim confiance : effectué.")

apport = len(stock_commerce_alim_actif_alim_confiance)-len(stock_commerce_alim_actif) 
print("Il y a un apport de données pour", apport, "établissements via la base alim confiance")

Téléchargement du fichier alim-confiance : effectué.
Importation du fichier CSV alim-confiance : effectué.
Jointure entre le fichier des commerces alimentaires actifs et alim confiance : effectué.
Il y a un apport de données pour 485 établissements via la base alim confiance


**On obtient un fichier avec les commerces alimentaires de la base SIRENE 3.11 et les commerces alimentaires issus de la base de donnée Alim Confiance**

## **Apport des données de l'Agence Bio à SIRENE 3.11**

In [4]:
# On requête directement pour télécharger le fichier issu de l'Agence Bio et le stocker dans le répertoire sirene 3.11

path_agence_bio = Path().home().joinpath(f"Documents/BDD sirene/sirene 3.11/operateursbio_{version}.csv")
url_agence_bio = "https://www.data.gouv.fr/fr/datasets/r/657789db-d349-4554-aef6-eabde4bd1c57"  

response_agence_bio = requests.get(url_agence_bio, stream=True)

with open(path_agence_bio, "wb") as file:
    for chunk in response_agence_bio.iter_content(chunk_size=1024):

        file.write(chunk) 

print("Téléchargement du fichier de l'Agence Bio : effectué.")

# On importe le fichier téléchargé
agence_bio = pl.read_csv(path_agence_bio.parent.joinpath(f"operateursbio_{version}.csv"), separator=";", ignore_errors=True, encoding='latin')
agence_bio = agence_bio.to_pandas()

print("Importation du fichier CSV de l'Agence Bio : effectué.")

agence_bio_distri = agence_bio[agence_bio['ACTIVITES'].str.contains("Distribution", na = False)] # On cherche les entités qui distribuent de l'alimentaire 
agence_bio_distri = agence_bio_distri.rename(columns={"SIRET" : "siret", "DENOMINATION": "enseigne1Etablissement", "CODE POSTAL SIEGE SOCIAL" : "codePostalEtablissement", "DATEENGAGEMENT" : "dateCreationEtablissement"}) # On renomme les variables pour que la jointure puisse être opérante

print("Création du fichier des distributeurs alimentaires issus de l'Agence Bio : effectué.")

stock_commerce_alim_actif_agence_bio = pd.merge(stock_commerce_alim_actif, agence_bio_distri, on="siret", how="left") # On fusionne les fichiers

print("Jointure entre les commerces alimentaires actifs et l'Agence Bio : effectué.")

doublon = stock_commerce_alim_actif_agence_bio["siret"].duplicated().sum() # On vérifie qu'il n'y ait pas de doublon
stock_commerce_alim_actif_agence_bio = stock_commerce_alim_actif_agence_bio.drop_duplicates("siret", keep='last') # On élimine les doublons en gardant les entités les plus récemments ajoutées

print("Traitement des doublons : effectué.")

Téléchargement du fichier de l'Agence Bio : effectué.
Importation du fichier CSV de l'Agence Bio : effectué.
Création du fichier des distributeurs alimentaires issus de l'Agence Bio : effectué.
Jointure entre les commerces alimentaires actifs et l'Agence Bio : effectué.
Traitement des doublons : effectué.


**On obtient un fichier comprenant les commerces alimentaires issus de la base SIRENE 3.11 et les commerces alimentaires issus de la base de données de l'Agence Bio**

## **Création d'un fichier comprenant SIRENE 3.11, Alim Confiance et l'Agence Bio**

In [5]:
# On fusionne les fichiers 

stock_commerce_enrichi =  pd.merge(stock_commerce_alim_actif_alim_confiance, agence_bio_distri, on="siret", how="left")

print("Jointure entre le fichier des commerces alimentaires actifs augmenté d'alim confiance avec le fichier des distributeurs de l'Agence Bio : effectué.")

Jointure entre le fichier des commerces alimentaires actifs augmenté d'alim confiance avec le fichier des distributeurs de l'Agence Bio : effectué.


## **On filtre les établissements fermés d'après le BODACC**

In [6]:
# On requête directement pour télécharger le fichier Bodacc des annonces de radiations et le stocker dans le répertoire sirene 3.11

url_bodacc = "https://bodacc-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/annonces-commerciales/exports/csv"
path_bodacc = Path().home().joinpath(f"Documents/BDD sirene/sirene 3.11/bodacc_{version}.csv")
params = {
    "refine": ["typeavis:'annonce'", "familleavis:'radiation'"],
    "lang": "fr",
    "select" : ["id", "dateparution", "familleavis", "ville", "numerodepartement", "departement_nom_officiel", "region_code", "region_nom_officiel","tribunal", "cp", "commercant", "registre", "jugement"], # On filtre les colonnes nécessaires à la jointure et à la compréhension du fichier
    "timezone": "Europe/Berlin"
}
response_bodacc = requests.get(url_bodacc, params=params, stream=True)

with open(path_bodacc, mode="wb") as file:
    for chunk in response_bodacc.iter_content(chunk_size=10 * 1024):
        file.write(chunk)

print("Téléchargement du fichier Bodacc des annonces de radiations : effectué.")

bodacc = pl.read_csv(path_bodacc, separator=";",  schema_overrides={"numerodepartement": pl.datatypes.String, "cp": pl.datatypes.String}) 
bodacc = bodacc.to_pandas() 

print("Importation du fichier CSV : effectué.")


# La variable registre est composée du même code siren écrit de deux manières différentes. On recrée une variable pour garder le code siren utile à la jointure. 
bodacc[["siren", "siren_bis"]] = bodacc.registre.str.split(pat=",", expand=True)
bodacc = bodacc.drop(["registre", "siren_bis"], axis=1)
bodacc = bodacc.replace('\[','',regex=True)
bodacc = bodacc.replace("\''",'',regex=True)
bodacc["siren"] = bodacc["siren"].str.strip("''")
stock_commerce_enrichi["siren"] = stock_commerce_enrichi["siren"].astype(str)

print("Modification de la variable siren pour jointure : effectué.")


# Jointure entre le fichier SIRENE enrichi et le BODACC & élimination des commerces alimentaires recensés comme radiés.
stock_commerce_alimentaire_bodacc = pd.merge(stock_commerce_enrichi, bodacc, on="siren", how='left')

def radiation_bodacc(x): 
    if x == "radiation" :
        return "Fermé"
    else :
        return "Ouvert"

stock_commerce_alimentaire_bodacc["etat_etablissement_bodacc"] = stock_commerce_alimentaire_bodacc["familleavis"].map(radiation_bodacc, na_action=None)

nb_fermeture = len(stock_commerce_alimentaire_bodacc.loc[stock_commerce_alimentaire_bodacc["etat_etablissement_bodacc"] == "Fermé"])
print("Il y a eu",nb_fermeture,"fermetures de commerces alimentaires (",version,")")

cols_object = ["siret", "siren", "nic"]
stock_commerce_alimentaire_bodacc[cols_object] = stock_commerce_alimentaire_bodacc[cols_object].astype(object, errors='ignore')

cols_object = ["trancheEffectifsEtablissement", "anneeEffectifsEtablissement", "numeroVoieEtablissement"]
stock_commerce_alimentaire_bodacc[cols_object] = stock_commerce_alimentaire_bodacc[cols_object].astype(int, errors='ignore')

stock_commerce_alimentaire_bodacc = stock_commerce_alimentaire_bodacc.drop('commercant', axis=1)

stock_commerce_alimentaire_bodacc.to_csv(f"stock_commerce_alimentaire_bodacc_{version_jour}.csv")

print("Création du fichier des commerces alimentaires actifs enrichi par les données de l'Agence Bio et Alim Confiance : effectué.")

<>:28: SyntaxWarning: invalid escape sequence '\['
<>:28: SyntaxWarning: invalid escape sequence '\['
C:\Users\qcreurer\AppData\Local\Temp\ipykernel_20184\1504592547.py:28: SyntaxWarning: invalid escape sequence '\['
  bodacc = bodacc.replace('\[','',regex=True)


Téléchargement du fichier Bodacc des annonces de radiations : effectué.
Importation du fichier CSV : effectué.
Modification de la variable siren pour jointure : effectué.
Il y a eu 8259 fermetures de commerces alimentaires ( February_2025 )
Création du fichier des commerces alimentaires actifs enrichi par les données de l'Agence Bio et Alim Confiance : effectué.
